In [1]:
from loader import load_classification_dataset
import simple_train, autoencoder_pretrain, configurator

import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split

from functools import partial

ds_path = os.path.join(os.getcwd(), 'datasets')

In [2]:
def print_accs(r):
    r = np.array(r)
    m = np.mean(r)
    std = np.std(r)
    print()
    print(f'Accuracy | Mean: {m:.2f} - Std: {std:.2f} | Max: {np.max(r):.2f}')

In [7]:
epochs = 200
verbose = 0

train_model, pretrain_model = next(iter(configurator.lstm_models()))

#for ds_name in os.listdir(ds_path):
for ds_name in ['UWaveGestureLibrary']:
    print(f'--- Dataset: {ds_name}')
    print()

    ds_load_result = load_classification_dataset(ds_path, ds_name)
    train_ds, test_ds, preprocessing_layer, label_encoder = ds_load_result

    _x_train, _y_train = train_ds
    x_test, y_test = test_ds

    _x_train = preprocessing_layer(_x_train).numpy()
    x_test = preprocessing_layer(x_test).numpy()

    _y_train = label_encoder.transform(_y_train)
    y_test = label_encoder.transform(y_test)

    x_train, x_val, y_train, y_val = train_test_split(
    _x_train, _y_train, test_size = 0.2, random_state = 42, shuffle=True)

    n_classes = len(label_encoder.classes_)

    print(f'Simple Train >')
    print()

    r = simple_train.train_evaluate(x_train, y_train, x_val, y_val, x_test, y_test, n_classes, model=train_model, verbose=verbose)

    print_accs(r)

    print(f'Pretrain and Fine Tune >')

    r = autoencoder_pretrain.train_evaluate(x_train, y_train, x_val, y_val, x_test, y_test, n_classes, model=pretrain_model, verbose=verbose)

    print_accs(r)

    print()
    print()

--- Dataset: UWaveGestureLibrary

Simple Train >


Accuracy | Mean: 0.82 - Std: 0.00 | Max: 0.82
Pretrain and Fine Tune >

Accuracy | Mean: 0.75 - Std: 0.00 | Max: 0.75


